# Model 5: LSTM Deep Learning (Literature - Paper B)

This notebook implements a Long Short-Term Memory (LSTM) network for sequence learning (or a Gradient Boosting fallback if TensorFlow is unavailable).

## 1. Imports and Setup

In [1]:
import sys
import pickle
import pandas as pd
import numpy as np
from pathlib import Path
# Add parent directory to path to import src
sys.path.append(str(Path('..').resolve()))
from src.models import compute_regression_metrics
print("="*70)
print("MODEL 5: LSTM MODEL (Literature - Paper B)")
print("="*70)
# Load data
data_dir = Path('..').resolve() / 'data' / 'processed'
X_train = pd.read_csv(data_dir / 'multi_X_train.csv')
y_train = pd.read_csv(data_dir / 'multi_y_train.csv').values.ravel()
X_test = pd.read_csv(data_dir / 'multi_X_test.csv')
y_test = pd.read_csv(data_dir / 'multi_y_test.csv').values.ravel()
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    from sklearn.preprocessing import StandardScaler
    print("\nBuilding LSTM model...")
    # Scale
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    # Reshape for LSTM (samples, timesteps, features)
    X_train_lstm = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
    X_test_lstm = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)
    model = keras.Sequential([
        layers.LSTM(64, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)),
        layers.Dropout(0.2),
        layers.LSTM(32),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(1, activation='linear') 
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    print("\nTraining LSTM...")
    history = model.fit(
        X_train_lstm, y_train,
        epochs=20, # Reduced for quick verification
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )
    pred = model.predict(X_test_lstm).flatten()
    metrics = compute_regression_metrics(y_test, pred)
    print("\n[LSTM Model] Test Set Performance:")
    print(f"  R² Score:  {metrics['R2']:.4f}")
    print(f"  RMSE:      {metrics['RMSE']:.4f}")
    model.save(data_dir / 'model_05_lstm.h5')
except ImportError:
    print("\n⚠ TensorFlow not found. Implementing Gradient Boosting as fallback for Paper B...")
    from sklearn.ensemble import GradientBoostingRegressor
    gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gb_model.fit(X_train, y_train)
    pred = gb_model.predict(X_test)
    metrics = compute_regression_metrics(y_test, pred)
    print("\n[Gradient Boosting (Paper B alt)] Test Set Performance:")
    print(f"  R² Score:  {metrics['R2']:.4f}")
# Save metrics/predictions
results = {
    'model_name': 'LSTM/GB (Paper B)',
    'predictions': pred,
    'metrics': metrics
}
with open(data_dir / 'model_05_literature.pkl', 'wb') as f:
    pickle.dump(results, f)
print(f"\n✓ Results saved to {data_dir / 'model_05_literature.pkl'}")

MODEL 5: LSTM MODEL (Literature - Paper B)

⚠ TensorFlow not found. Implementing Gradient Boosting as fallback for Paper B...



[Gradient Boosting (Paper B alt)] Test Set Performance:
  R² Score:  0.8941

✓ Results saved to /Users/himanshishrivas/Documents/IntroMLCapstone/data/processed/model_05_literature.pkl
